<15.광양시_건물정보> 데이터에서 <세부용도명>을 기준으로 아래에 해당하는 데이터를 추린 후 각각 전처리하고 합침  
1)아파트, 다가구주택, 다세대주택, 공동주택, 다중주택  
2)기타공공시설, 동사무소, 공공시설, 공공업무시설  
3)대학교, 대학, 쇼핑센터, 관광호텔, 호텔, 주차장  
4)기타문화및집회시설, 도서관, 종합병원,휴게소  

In [1]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from pandas import Series, DataFrame
import math
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


In [3]:
buildinfo = gpd.read_file("15.광양시_건물정보.geojson")
owner_land = gpd.read_file('14.광양시_소유지정보.geojson')

# 1. 각 항목별로 전처리하기

# 1) 아파트, 다가구주택, 다세대주택, 공동주택, 다중주택  
공공데이터포털에서 다운받은 <전라남도 광양시 공동주택 현황_20200929.csv> 데이터를 활용하여 각 주택에 대한 세대수를 통해 주차면수를 계산

In [4]:
공동주택현황 = pd.read_csv("전라남도 광양시 공동주택 현황_20200929.csv")
공동주택현황

,연번,아파트명,읍면동,위치,사업주체,층수,동수,세대수,사업승인,사용검사,유형,비고
0,1,주공칠성 ①②,광양읍,읍 칠성리 89-5,대한주택공사,5,25,1040,1986-08-12,1987-10-14,분양,NaN
1,2,주공칠성 ③,광양읍,읍 칠성리 89-5,대한주택공사,5,17,590,1987-05-26,1990-07-01,분양,NaN
2,3,삼보타워맨션,광양읍,읍 칠성리 524-7,㈜삼보건설,15,1,99,1991-10-15,1993-06-15,분양,NaN
3,4,신광맨션,광양읍,읍 칠성리 41,신광건설㈜,13,1,50,1990-06-01,1991-09-06,분양,NaN
4,5,영진맨션,광양읍,읍 칠성리 528,영진건설㈜,5,1,45,1989-06-15,1990-01-11,분양,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,동백아파트,금호동,금호동 714,포항종합제철㈜,5,14,365,1983-02-04,1988-01-05,분양,NaN
185,186,동백아파트,금호동,금호동 715,포항종합제철㈜,5,14,365,1983-02-04,1988-01-05,분양,NaN
186,187,장미아파트,금호동,금호동 720,포항종합제철㈜,5,26,661,1984-06-05,1986-10-14,분양,NaN
187,188,장미아파트,금호동,금호동 725,포항종합제철㈜,5,26,661,1984-06-05,1986-10-14,분양,NaN


In [5]:
공동주택현황['건물당 세대수'] = 공동주택현황['세대수']/공동주택현황['동수']
공동주택현황 = 공동주택현황.astype({'건물당 세대수':'int'})
공동주택현황

,연번,아파트명,읍면동,위치,사업주체,층수,동수,세대수,사업승인,사용검사,유형,비고,건물당 세대수
0,1,주공칠성 ①②,광양읍,읍 칠성리 89-5,대한주택공사,5,25,1040,1986-08-12,1987-10-14,분양,NaN,41
1,2,주공칠성 ③,광양읍,읍 칠성리 89-5,대한주택공사,5,17,590,1987-05-26,1990-07-01,분양,NaN,34
2,3,삼보타워맨션,광양읍,읍 칠성리 524-7,㈜삼보건설,15,1,99,1991-10-15,1993-06-15,분양,NaN,99
3,4,신광맨션,광양읍,읍 칠성리 41,신광건설㈜,13,1,50,1990-06-01,1991-09-06,분양,NaN,50
4,5,영진맨션,광양읍,읍 칠성리 528,영진건설㈜,5,1,45,1989-06-15,1990-01-11,분양,NaN,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,185,동백아파트,금호동,금호동 714,포항종합제철㈜,5,14,365,1983-02-04,1988-01-05,분양,NaN,26
185,186,동백아파트,금호동,금호동 715,포항종합제철㈜,5,14,365,1983-02-04,1988-01-05,분양,NaN,26
186,187,장미아파트,금호동,금호동 720,포항종합제철㈜,5,26,661,1984-06-05,1986-10-14,분양,NaN,25
187,188,장미아파트,금호동,금호동 725,포항종합제철㈜,5,26,661,1984-06-05,1986-10-14,분양,NaN,25


In [6]:
address = []
for i in range(len(공동주택현황)):
    address.append(공동주택현황['위치'][i].split(" ")[1:])
print(address)

[['칠성리', '89-5'], ['칠성리', '89-5'], ['칠성리', '524-7'], ['칠성리', '41'], ['칠성리', '528'], ['칠성리', '520-5'], ['칠성리', '899-2'], ['칠성리', '520-3'], ['칠성리', '894-7'], ['칠성리', '896-3'], ['칠성리', '914-1'], ['인서리', '480'], ['인서리', '103-1'], ['목성리', '1000'], ['목성리', '555'], ['목성리', '1008-1'], ['덕례리', '1725'], ['덕례리', '1670-5'], ['구산리', '137-2'], ['우산리', '563-1'], ['우산리', '407-6'], ['1366'], ['552-2'], ['1301-1'], ['1354'], ['1294'], ['1434'], ['1416'], ['1418'], ['1356-2'], ['1376'], ['757-2'], ['755-2'], ['757-1'], ['313'], ['775-1'], ['775-2'], ['755-1'], ['1320-41'], ['826'], ['825'], ['712'], ['624-13'], ['724'], ['401'], ['덕례리', '1735-1'], ['1681'], ['덕례리', '1662-1'], ['덕례리', '1082-1'], ['1686'], ['1663'], ['덕례리', '1687-1'], ['1682'], ['덕례리', '1755-1'], ['1691'], ['칠성리', '952'], ['용강리', '892'], ['용강리', '893'], ['1019'], ['덕례리', '1695-1'], ['구산리', '762-2'], ['1655'], ['1665'], ['덕례리', '1688-1'], ['인서리', '136'], ['1696-4'], ['1597'], ['1046'], ['1054'], ['1030'], ['1055'], ['1031'], ['850-1'], ['10

## (1) 세부용도명 = 아파트

In [7]:
아파트 = buildinfo[buildinfo['세부용도명']=='아파트']
아파트.reset_index(drop=True,inplace=True)

아파트['세대수'] = np.nan
for i in range(len(아파트)):
    for j in range(len(address)):
        for k in range(len(address[j])):
            if 아파트['지번'][i] == address[j][k]:
                아파트['세대수'][i] = 공동주택현황['건물당 세대수'][j]
                
아파트 = 아파트[아파트['세대수'].notnull()]
아파트.reset_index(drop=True,inplace=True)

<주택건설기준 등에 관한 규정> 제 27조  
주차면수 = 전용면적 X (세대수/75)  
(전용면적 100m^2로 가정)

다만, 전용면적 60m^2이상일 경우 세대 당 주차면수가 1이상이어야 함

In [8]:
아파트['주차면수'] = np.nan
for i in range(len(아파트)):
    if 102*(아파트['세대수'][i]/75) < 아파트['세대수'][i]:
        아파트['주차면수'][i] = 아파트['세대수'][i]
    else:
        아파트['주차면수'][i] = 102*(아파트['세대수'][i]/75)
아파트.drop(['세대수'], axis='columns', inplace=True)
아파트['주차면수']=round(아파트['주차면수'],0)
아파트

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry,주차면수
0,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,106동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69997 34.94371, 127.69964 ...",61.0
1,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,110동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69946 34.94304, 127.69978 ...",61.0
2,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,108동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69997 34.94333, 127.70058 ...",61.0
3,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,107동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69997 34.94356, 127.70046 ...",61.0
4,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,109동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.70001 34.94307, 127.70016 ...",61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,4623025035117550001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1755-1,대림아파트,102동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,46.8,17.0,1.0,"MULTIPOLYGON (((127.56443 34.96894, 127.56518 ...",207.0
420,4623025035117550001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1755-1,대림아파트,106동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,46.8,17.0,1.0,"MULTIPOLYGON (((127.56618 34.96906, 127.56616 ...",207.0
421,4623025035117550001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1755-1,대림아파트,103동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,46.8,17.0,1.0,"MULTIPOLYGON (((127.56426 34.96898, 127.56423 ...",207.0
422,4623033024113660000,4623033024,전라남도 광양시 옥곡면 신금리,1,일반,1366,화진그린맨션,None,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,6.0,1.0,"MULTIPOLYGON (((127.69997 34.99058, 127.69991 ...",65.0


## (2) 세부용도명 = 다중주택

1)50m^2 < 시설면적 < 150m^2 인 경우 --> 1대  
2)시설면적 > 150m^ 인 경우--> 1 + (연면적 - 150)/100

In [9]:
다중주택 = buildinfo[buildinfo['세부용도명']=='다중주택']
다중주택.reset_index(drop=True,inplace=True)
다중주택

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조코드,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry
0,4623025032104690006,4623025032,전라남도 광양시 광양읍 인서리,1,일반,469-6,None,제1동,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01002,다중주택,7.95,3.0,0.0,"MULTIPOLYGON (((127.58678 34.96885, 127.58688 ..."
1,4623025035105440006,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,544-6,D동,None,0,주건축물,...,11,벽돌구조,01000,단독주택,01002,다중주택,4.00,1.0,0.0,"MULTIPOLYGON (((127.57149 34.95945, 127.57133 ..."
2,4623025035106710001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,671-1,None,None,0,주건축물,...,11,벽돌구조,01000,단독주택,01002,다중주택,8.20,2.0,0.0,"MULTIPOLYGON (((127.56663 34.95750, 127.56681 ..."
3,4623025035109170001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,917-1,None,None,0,주건축물,...,32,경량철골구조,01000,단독주택,01002,다중주택,4.51,1.0,0.0,"MULTIPOLYGON (((127.56740 34.96671, 127.56717 ..."
4,4623025035116760001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1676-1,None,제1동,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01002,다중주택,9.40,3.0,0.0,"MULTIPOLYGON (((127.57142 34.96523, 127.57145 ..."
5,4623025035117290014,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1729-14,None,None,0,주건축물,...,11,벽돌구조,01000,단독주택,01002,다중주택,0.00,2.0,0.0,"MULTIPOLYGON (((127.57436 34.96359, 127.57455 ..."
6,4623025035117290015,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1729-15,None,None,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01002,다중주택,0.00,2.0,0.0,"MULTIPOLYGON (((127.57438 34.96348, 127.57441 ..."
7,4623025035117340009,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1734-9,None,None,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01002,다중주택,7.20,2.0,0.0,"MULTIPOLYGON (((127.57629 34.96330, 127.57631 ..."
8,4623025035117360009,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1736-9,None,None,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01002,다중주택,7.00,2.0,0.0,"MULTIPOLYGON (((127.57744 34.96404, 127.57744 ..."
9,4623025035117410003,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1741-3,None,None,0,주건축물,...,11,벽돌구조,01000,단독주택,01002,다중주택,0.00,2.0,0.0,"MULTIPOLYGON (((127.57933 34.96453, 127.57949 ..."


In [10]:
다중주택['주차면수'] = np.nan
for i in range(len(다중주택)):
    if 다중주택['건물연면적'][i] < 150:
        다중주택['주차면수'][i] = 1
    else:
        다중주택['주차면수'][i] = 1 + (다중주택['건물연면적'][i] - 150)/100
다중주택['주차면수']=round(다중주택['주차면수'],0)
다중주택

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry,주차면수
0,4623025032104690006,4623025032,전라남도 광양시 광양읍 인서리,1,일반,469-6,None,제1동,0,주건축물,...,철근콘크리트구조,01000,단독주택,01002,다중주택,7.95,3.0,0.0,"MULTIPOLYGON (((127.58678 34.96885, 127.58688 ...",2.0
1,4623025035105440006,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,544-6,D동,None,0,주건축물,...,벽돌구조,01000,단독주택,01002,다중주택,4.00,1.0,0.0,"MULTIPOLYGON (((127.57149 34.95945, 127.57133 ...",1.0
2,4623025035106710001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,671-1,None,None,0,주건축물,...,벽돌구조,01000,단독주택,01002,다중주택,8.20,2.0,0.0,"MULTIPOLYGON (((127.56663 34.95750, 127.56681 ...",3.0
3,4623025035109170001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,917-1,None,None,0,주건축물,...,경량철골구조,01000,단독주택,01002,다중주택,4.51,1.0,0.0,"MULTIPOLYGON (((127.56740 34.96671, 127.56717 ...",1.0
4,4623025035116760001,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1676-1,None,제1동,0,주건축물,...,철근콘크리트구조,01000,단독주택,01002,다중주택,9.40,3.0,0.0,"MULTIPOLYGON (((127.57142 34.96523, 127.57145 ...",2.0
5,4623025035117290014,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1729-14,None,None,0,주건축물,...,벽돌구조,01000,단독주택,01002,다중주택,0.00,2.0,0.0,"MULTIPOLYGON (((127.57436 34.96359, 127.57455 ...",2.0
6,4623025035117290015,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1729-15,None,None,0,주건축물,...,철근콘크리트구조,01000,단독주택,01002,다중주택,0.00,2.0,0.0,"MULTIPOLYGON (((127.57438 34.96348, 127.57441 ...",2.0
7,4623025035117340009,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1734-9,None,None,0,주건축물,...,철근콘크리트구조,01000,단독주택,01002,다중주택,7.20,2.0,0.0,"MULTIPOLYGON (((127.57629 34.96330, 127.57631 ...",2.0
8,4623025035117360009,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1736-9,None,None,0,주건축물,...,철근콘크리트구조,01000,단독주택,01002,다중주택,7.00,2.0,0.0,"MULTIPOLYGON (((127.57744 34.96404, 127.57744 ...",2.0
9,4623025035117410003,4623025035,전라남도 광양시 광양읍 덕례리,1,일반,1741-3,None,None,0,주건축물,...,벽돌구조,01000,단독주택,01002,다중주택,0.00,2.0,0.0,"MULTIPOLYGON (((127.57933 34.96453, 127.57949 ...",1.0


## (3) 세부용도명 = 다가구주택, 다세대주택, 공동주택

In [11]:
다가구_다세대_공동 = buildinfo[buildinfo['세부용도명'].isin(['다세대주택', '다가구주택', '공동주택'])]
다가구_다세대_공동.reset_index(drop=True,inplace=True)
다가구_다세대_공동

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조코드,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry
0,4623010600112950002,4623010600,전라남도 광양시 중동,1,일반,1295-2,None,None,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01003,다가구주택,13.85,4.0,1.0,"MULTIPOLYGON (((127.69870 34.94394, 127.69870 ..."
1,4623010600112960003,4623010600,전라남도 광양시 중동,1,일반,1296-3,None,None,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01003,다가구주택,7.65,2.0,0.0,"MULTIPOLYGON (((127.69868 34.94235, 127.69868 ..."
2,4623010600112970002,4623010600,전라남도 광양시 중동,1,일반,1297-2,None,제1동,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01003,다가구주택,13.15,4.0,0.0,"MULTIPOLYGON (((127.69937 34.94264, 127.69950 ..."
3,4623010600112970003,4623010600,전라남도 광양시 중동,1,일반,1297-3,None,제1동,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01003,다가구주택,13.30,4.0,0.0,"MULTIPOLYGON (((127.69955 34.94264, 127.69968 ..."
4,4623010600112970008,4623010600,전라남도 광양시 중동,1,일반,1297-8,None,제1동,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01003,다가구주택,12.55,4.0,0.0,"MULTIPOLYGON (((127.69936 34.94246, 127.69950 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,4623036024111820002,4623036024,전라남도 광양시 다압면 금천리,1,일반,1182-2,None,주건축물제5동,0,주건축물,...,32,경량철골구조,01000,단독주택,01003,다가구주택,3.80,1.0,0.0,"MULTIPOLYGON (((127.63993 35.14673, 127.63993 ..."
747,4623036024117240000,4623036024,전라남도 광양시 다압면 금천리,1,일반,1724,None,None,0,주건축물,...,31,일반철골구조,01000,단독주택,01003,다가구주택,8.80,2.0,0.0,"MULTIPOLYGON (((127.64455 35.15768, 127.64450 ..."
748,4623036024122660003,4623036024,전라남도 광양시 다압면 금천리,1,일반,2266-3,None,제1동,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01003,다가구주택,7.00,2.0,0.0,"MULTIPOLYGON (((127.62967 35.17649, 127.62969 ..."
749,4623036024122730000,4623036024,전라남도 광양시 다압면 금천리,1,일반,2273,None,나동,0,주건축물,...,31,일반철골구조,01000,단독주택,01003,다가구주택,0.00,3.0,0.0,"MULTIPOLYGON (((127.62952 35.17625, 127.62952 ..."


In [12]:
다가구_다세대_공동['세대수'] = np.nan
for i in range(len(다가구_다세대_공동)):
    for j in range(len(address)):
        for k in range(len(address[j])):
            if 다가구_다세대_공동['지번'][i] == address[j][k]:
                다가구_다세대_공동['세대수'][i] = 공동주택현황['건물당 세대수'][j]
                
다가구_다세대_공동 = 다가구_다세대_공동[다가구_다세대_공동['세대수'].notnull()]
다가구_다세대_공동.reset_index(drop=True,inplace=True)
다가구_다세대_공동

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry,세대수
0,4623010600113580002,4623010600,전라남도 광양시 중동,1,일반,1358-2,None,None,0,주건축물,...,벽돌구조,02000,공동주택,02003,다세대주택,16.00,4.0,0.0,"MULTIPOLYGON (((127.69366 34.94203, 127.69366 ...",4.0
1,4623010600113610020,4623010600,전라남도 광양시 중동,1,일반,1361-20,송림빌라,None,0,주건축물,...,철근콘크리트구조,02000,공동주택,02003,다세대주택,15.00,5.0,0.0,"MULTIPOLYGON (((127.69024 34.93947, 127.69028 ...",8.0
2,4623010600113680001,4623010600,전라남도 광양시 중동,1,일반,1368-1,None,None,0,주건축물,...,철근콘크리트구조,02000,공동주택,02003,다세대주택,10.64,3.0,0.0,"MULTIPOLYGON (((127.68627 34.94092, 127.68627 ...",6.0
3,4623010600113680007,4623010600,전라남도 광양시 중동,1,일반,1368-7,다세대주택,None,0,주건축물,...,기타조적구조,02000,공동주택,02003,다세대주택,8.60,3.0,0.0,"MULTIPOLYGON (((127.68611 34.94071, 127.68610 ...",4.0
4,4623010600113710004,4623010600,전라남도 광양시 중동,1,일반,1371-4,None,None,0,주건축물,...,기타조적구조,01000,단독주택,01003,다가구주택,7.10,2.0,0.0,"MULTIPOLYGON (((127.68600 34.94079, 127.68600 ...",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,라동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73728 35.05858, 127.73723 ...",3.0
77,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,다동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73735 35.05860, 127.73731 ...",3.0
78,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,사동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73670 35.05823, 127.73677 ...",3.0
79,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,자동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73677 35.05789, 127.73689 ...",3.0


<주택건설기준 등에 관한 규정> 제 27조  
주차면수 = 전용면적 X (세대수/95)  
(전용면적 60m^2로 가정)

다만, 전용면적 60m^2이하일 경우 세대 당 주차면수가 0.7대이상이어야 함

In [13]:
다가구_다세대_공동['주차면수'] = np.nan
for i in range(len(다가구_다세대_공동)):
    if 60*(다가구_다세대_공동['세대수'][i]/95) < 다가구_다세대_공동['세대수'][i] * 0.7:
        다가구_다세대_공동['주차면수'][i] = 다가구_다세대_공동['세대수'][i] * 0.7
    else:
        다가구_다세대_공동['주차면수'][i] = 60*(다가구_다세대_공동['세대수'][i]/95)       
다가구_다세대_공동['주차면수']=round(다가구_다세대_공동['주차면수'],0)
다가구_다세대_공동.drop(['세대수'], axis='columns', inplace=True)
다가구_다세대_공동

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry,주차면수
0,4623010600113580002,4623010600,전라남도 광양시 중동,1,일반,1358-2,None,None,0,주건축물,...,벽돌구조,02000,공동주택,02003,다세대주택,16.00,4.0,0.0,"MULTIPOLYGON (((127.69366 34.94203, 127.69366 ...",3.0
1,4623010600113610020,4623010600,전라남도 광양시 중동,1,일반,1361-20,송림빌라,None,0,주건축물,...,철근콘크리트구조,02000,공동주택,02003,다세대주택,15.00,5.0,0.0,"MULTIPOLYGON (((127.69024 34.93947, 127.69028 ...",6.0
2,4623010600113680001,4623010600,전라남도 광양시 중동,1,일반,1368-1,None,None,0,주건축물,...,철근콘크리트구조,02000,공동주택,02003,다세대주택,10.64,3.0,0.0,"MULTIPOLYGON (((127.68627 34.94092, 127.68627 ...",4.0
3,4623010600113680007,4623010600,전라남도 광양시 중동,1,일반,1368-7,다세대주택,None,0,주건축물,...,기타조적구조,02000,공동주택,02003,다세대주택,8.60,3.0,0.0,"MULTIPOLYGON (((127.68611 34.94071, 127.68610 ...",3.0
4,4623010600113710004,4623010600,전라남도 광양시 중동,1,일반,1371-4,None,None,0,주건축물,...,기타조적구조,01000,단독주택,01003,다가구주택,7.10,2.0,0.0,"MULTIPOLYGON (((127.68600 34.94079, 127.68600 ...",4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,라동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73728 35.05858, 127.73723 ...",2.0
77,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,다동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73735 35.05860, 127.73731 ...",2.0
78,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,사동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73670 35.05823, 127.73677 ...",2.0
79,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,자동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.40,2.0,0.0,"MULTIPOLYGON (((127.73677 35.05789, 127.73689 ...",2.0


## (4) 합치기

In [14]:
주거 = pd.concat([아파트, 다중주택, 다가구_다세대_공동], ignore_index=True)
주거

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry,주차면수
0,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,106동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69997 34.94371, 127.69964 ...",61.0
1,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,110동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69946 34.94304, 127.69978 ...",61.0
2,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,108동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69997 34.94333, 127.70058 ...",61.0
3,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,107동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.69997 34.94356, 127.70046 ...",61.0
4,4623010600112940000,4623010600,전라남도 광양시 중동,1,일반,1294,중동아파트,109동,0,주건축물,...,철근콘크리트구조,02000,공동주택,02001,아파트,0.0,5.0,0.0,"MULTIPOLYGON (((127.70001 34.94307, 127.70016 ...",61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,라동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.4,2.0,0.0,"MULTIPOLYGON (((127.73728 35.05858, 127.73723 ...",2.0
516,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,다동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.4,2.0,0.0,"MULTIPOLYGON (((127.73735 35.05860, 127.73731 ...",2.0
517,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,사동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.4,2.0,0.0,"MULTIPOLYGON (((127.73670 35.05823, 127.73677 ...",2.0
518,4623036021104070004,4623036021,전라남도 광양시 다압면 신원리,1,일반,407-4,평화를여는마을,자동,0,주건축물,...,경량철골구조,02000,공동주택,02003,다세대주택,7.4,2.0,0.0,"MULTIPOLYGON (((127.73677 35.05789, 127.73689 ...",2.0


## (5) category컬럼 & 소유주체 컬럼 추가

In [15]:
주거['category'] = '주거'
주거['lon']=주거['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
주거['lat']=주거['geometry'].centroid.y
주거['geometry'] = 주거.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)

In [41]:
주거1 = 주거.loc[:130]
주거2 = 주거.loc[130:260]
주거3 = 주거.loc[260:390]
주거4 = 주거.loc[390:]

주거2.reset_index(drop=True, inplace=True)
주거3.reset_index(drop=True, inplace=True)
주거4.reset_index(drop=True, inplace=True)

In [67]:
주거1['소유주체'] = np.nan
for i in range(len(주거1)):
    for j in range(len(owner_land)):
        if 주거1.geometry[i].within(owner_land.geometry[j]) == True:
            주거1['소유주체'][i]=owner_land.소유구분명[j]       

In [ ]:
주거2['소유주체'] = np.nan
for i in range(len(주거2)):
    for j in range(len(owner_land)):
        if 주거2.geometry[i].within(owner_land.geometry[j]) == True:
            주거2['소유주체'][i]=owner_land.소유구분명[j]

In [50]:
주거3['소유주체'] = np.nan
for i in range(len(주거3)):
    for j in range(len(owner_land)):
        if 주거3.geometry[i].within(owner_land.geometry[j]) == True:
            주거3['소유주체'][i]=owner_land.소유구분명[j]
            

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,세부용도명,건물높이,지상층수,지하층수,geometry,주차면수,category,lon,lat,소유주체
0,4623011000107250000,4623011000,전라남도 광양시 금호동,1,일반,725,장미아파트,13동,0,주건축물,...,아파트,0.0,5.0,0.0,POINT (127.73307 34.94101),34.0,주거,127.733067,34.941011,NaN
1,4623011000107250000,4623011000,전라남도 광양시 금호동,1,일반,725,장미아파트,9동,0,주건축물,...,아파트,0.0,5.0,0.0,POINT (127.73367 34.94165),34.0,주거,127.733674,34.941654,NaN
2,4623011000108250000,4623011000,전라남도 광양시 금호동,1,일반,825,초원아파트,6동,0,주건축물,...,아파트,40.7,5.0,0.0,POINT (127.71971 34.93776),38.0,주거,127.719707,34.937759,NaN
3,4623011000108250000,4623011000,전라남도 광양시 금호동,1,일반,825,초원아파트,7동,0,주건축물,...,아파트,40.7,5.0,0.0,POINT (127.72040 34.93822),38.0,주거,127.720397,34.938223,NaN
4,4623011000108250000,4623011000,전라남도 광양시 금호동,1,일반,825,초원아파트,2동,0,주건축물,...,아파트,40.7,5.0,0.0,POINT (127.72103 34.93832),38.0,주거,127.721025,34.938322,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,4623025032101340001,4623025032,전라남도 광양시 광양읍 인서리,1,일반,134-1,평화그린빌라,None,0,주건축물,...,아파트,0.0,5.0,0.0,POINT (127.58318 34.97017),26.0,주거,127.583183,34.970175,NaN
127,4623025032104800000,4623025032,전라남도 광양시 광양읍 인서리,1,일반,480,부영아파트,104동,0,주건축물,...,아파트,0.0,5.0,0.0,POINT (127.58369 34.96752),67.0,주거,127.583690,34.967521,NaN
128,4623025032104800000,4623025032,전라남도 광양시 광양읍 인서리,1,일반,480,부영아파트,106동,0,주건축물,...,아파트,0.0,5.0,0.0,POINT (127.58372 34.96703),67.0,주거,127.583716,34.967028,NaN
129,4623025032104800000,4623025032,전라남도 광양시 광양읍 인서리,1,일반,480,부영아파트,105동,0,주건축물,...,아파트,0.0,5.0,0.0,POINT (127.58383 34.96735),67.0,주거,127.583832,34.967346,NaN


In [ ]:
주거4['소유주체'] = np.nan
for i in range(len(주거4)):
    for j in range(len(owner_land)):
        if 주거4.geometry[i].within(owner_land.geometry[j]) == True:
            주거4['소유주체'][i]=owner_land.소유구분명[j]

In [70]:
항목1 = pd.concat([주거1, 주거2,주거3,주거4], ignore_index=True)
항목1.to_csv("항목1.csv", index=False, encoding='cp949')

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

# 2) 기타공공시설, 동사무소, 공공시설, 공공업무시설 

2) 기타공공시설, 동사무소, 공공시설, 공공업무시설 --> 주요용도명이 모두 '제1종근린생활시설'

<부설주차장의 설치대상 시설물 종류 및 설치기준> 중 시설물 2번으로 계산

In [ ]:
항목2 = buildinfo[buildinfo['세부용도명'].isin(['기타공공시설','동사무소','공공시설','공공업무시설'])]
항목2.reset_index(drop=True,inplace=True)

항목2['주차면수'] = round(항목2['건물연면적'] / 200, 0)

항목2['category'] = '공공시설'
항목2['lon']=항목2['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
항목2['lat']=항목2['geometry'].centroid.y
항목2['geometry'] = 항목2.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)

항목2['소유주체'] = np.nan
for i in range(len(항목2)):
    for j in range(len(owner_land)):
        if 항목2.geometry[i].within(owner_land.geometry[j]) == True:
            항목2['소유주체'][i]=owner_land.소유구분명[j]

# 3)대학교, 대학, 쇼핑센터, 관광호텔, 호텔, 주차장   

In [ ]:
항목3 = buildinfo[buildinfo['세부용도명'].isin(['대학교','대학', '쇼핑센터', '관광호텔', '호텔'])]
항목3.reset_index(drop=True,inplace=True)
항목3['주차면수'] = 0

for i in range(len(항목3)):
    if 항목3['세부용도명'][i]=='대학교':
        항목3['주차면수'][i] = round(항목3['건물연면적'][i]/300)   #10번 문항으로 계산
    elif 항목3['세부용도명'][i]=='대학':
        항목3['주차면수'][i] = round(항목3['건물연면적'][i]/300)   #10번 문항으로 계산
    elif 항목3['세부용도명'][i]=='쇼핑센터':
        항목3['주차면수'][i] = round(항목3['건물연면적'][i]/150)   #2번 문항으로 계산
    elif 항목3['세부용도명'][i]=='관광호텔':
        항목3['주차면수'][i] = round(항목3['건물연면적'][i]/200)   #2번 문항으로 계산
    elif 항목3['세부용도명'][i]=='호텔':
        항목3['주차면수'][i] = round(항목3['건물연면적'][i]/200)   #3번 문항으로 계산

세부용도가 주차장인 경우 <2.광양시_주차장_공간정보>데이터를 전처리한 <광양시_주차장_공간정보_전처리>데이터를 활용하여 구했습니다.


In [ ]:
final_parkinglot = pd.read_csv('광양시_주차장_공간정보_전처리.csv')
parkinglot = building[building['세부용도명']=='주차장']

#주차장 면적의 자릿수를 구해서 s에 넣음
a = final_parkinglot['DGM_AR']
s = set()
for i in a:
    if str(i) == "nan":
        pass
    else:
        z = len(str(int(i)))
        s.add(z)
      
    
head =[]
#index 0~3까지는 각 면적의 자릿수+ 앞자리 별 개수, 4~7까지는 각 면적의 자릿수+ 앞자리 계신 값 들의 sum 
for i in range(len(s)+4):
    head.append([0,0,0,0,0,0,0,0,0,0])   #10X8 형태의 배열
    


In [ ]:
b = final_parkinglot['주차면수']

# n = 면적//주차면수 
def count_parkinglot(x, y):
    n = (x//y)
    return n

def head_index(num1, num2, x, y):
  #각각의 n값과, 개수 추가
    for i in range(10):
        if int(x) ==i:
            head[num1][i]+=1
            head[num2][i] +=y

count =0 
for i in a:
    if str(i) == "nan" or str(b[count])== "nan":
        pass
    else:
        if len(str(int(i))) == 2:
            num=count_parkinglot(i, b[count])
            head_index(0, 4, str(int(i))[0], num)
        elif len(str(int(i))) == 3:
            num=count_parkinglot(i, b[count])
            head_index(1, 5, str(int(i))[0], num)
        elif len(str(int(i))) == 4:
            num=count_parkinglot(i, b[count])
            head_index(2, 6, str(int(i))[0], num)
        elif len(str(int(i))) == 5:
            num=count_parkinglot(i, b[count])
            head_index(3, 7, str(int(i))[0], num)
    count+=1

In [ ]:
_count =0
_count1=3

head2 =[]

for i in range(4):
    head2.append([0,0,0,0,0,0,0,0,0,0])

for i in head[4:8]:
    for j in range(10):
        if head[_count][j]!=0:
            head2[_count][j]=i[j]//head[_count][j]
    _count+=1
    _count1+=1

In [ ]:
pp = parkinglot['건물건축면적']
s = set()
for i in pp:
    if str(i) == "nan":
        pass
    else:
        z = len(str(int(i)))
        s.add(z)

In [ ]:
count =0 
sum=0

for i in pp:
    if int(len(str(int(i))))==2:
        sum+=round(int(i)/11.728,0)
        count +=1

In [ ]:
pp = parkinglot['건물건축면적']
final = []

def count_parkinglot(x, y):
    for k in range(10):
        if int(str(x)[0]) == k:
            if int(y[k]) == 0:
                if len(str(int(x)))==2:
                    final.append(6)
                elif len(str(int(x)))==3:
                    final.append(17)
            else:
                n = int(x)//y[k]
                final.append(n)

for i in pp:
    if str(i) == "nan":
        final.append(None)
    else:
        if len(str(int(i))) == 2:
            num=count_parkinglot(i, head2[0])
        elif len(str(int(i))) == 3:
            num=count_parkinglot(i, head2[1])
    count+=1
    
parkinglot['주차면수']= final

In [ ]:
항목3['category'] = np.nan
for i in range(len(항목3)):
    if 항목3['세부용도명'][i] == '주차장':
        항목3['category'][i] = '주차장'
    elif (항목3['세부용도명'][i] == '대학교') or (항목3['세부용도명'][i] == '대학'):
        항목3['category'][i] = '대학교'
    elif 항목3['세부용도명'][i] == '쇼핑센터':
        항목3['category'][i] = '쇼핑센터'
    else:
        항목3['category'][i] = '호텔'

항목3['lon']=항목3['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
항목3['lat']=항목3['geometry'].centroid.y
항목3['geometry'] = 항목3.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)

항목3['소유주체'] = np.nan
for i in range(len(항목3)):
    for j in range(len(owner_land)):
        if 항목3.geometry[i].within(owner_land.geometry[j]) == True:
            항목3['소유주체'][i]=owner_land.소유구분명[j]
            
항목3

In [ ]:
항목3.to_csv("항목4.csv", index=False, encoding='cp949')

# 4) 휴게소, 기타문화및집회시설, 도서관, 종합병원

In [75]:
항목4 = buildinfo[buildinfo['세부용도명'].isin(['휴게소','기타문화및집회시설', '도서관', '종합병원'])]
항목4.reset_index(drop=True,inplace=True)
항목4['주차면수'] = 0

#각 용도에 따라 주차면수 계산
for i in range(len(항목4)):
    if 항목4['주요용도명'][i] == '문화및집회시설': #2번 문항으로 계산
        num = 항목4['건물연면적'][i]//150
        항목4['주차면수'][i] = round(num,0) 
    elif 항목4['주요용도명'][i] == '교육연구시설' : #10번 문항으로 계산
        num = 항목4['건물연면적'][i]//300
        항목4['주차면수'][i] = round(num,0)
    elif 항목4['주요용도명'][i] == '관광휴게시설' : #10번 문항으로 계산
        num = 항목4['건물연면적'][i]//300
        항목4['주차면수'][i] = round(num,0)
    else:   #2번 문항으로 계산(의료시설 1개)
        num = 항목4['건물연면적'][i]//150 
        항목4['주차면수'][i] = round(num,0)

In [ ]:
항목4['category'] = np.nan
for i in range(len(항목4)):
    if 항목4['세부용도명'][i] == '휴게소':
        항목4['category'][i] = '휴게소'
    elif 항목4['세부용도명'][i] == '기타문화및집회시설':
        항목4['category'][i] = '기타문화및집회시설'
    elif 항목4['세부용도명'][i] == '도서관':
        항목4['category'][i] = '도서관'
    else:
        항목4['category'][i] = '종합병원'
        

항목4['lon']=항목4['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
항목4['lat']=항목4['geometry'].centroid.y
항목4['geometry'] = 항목4.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)

항목4['소유주체'] = np.nan
for i in range(len(항목4)):
    for j in range(len(owner_land)):
        if 항목4.geometry[i].within(owner_land.geometry[j]) == True:
            항목4['소유주체'][i]=owner_land.소유구분명[j]
            
항목4

In [77]:
항목4.to_csv("항목4(1).csv", index=False, encoding='cp949')

----------

# 2. 건물정보 전체 합치고 필요없는 컬럼 삭제

In [87]:
항목1 = pd.read_csv("항목1.csv", encoding='utf-8')
항목2 = pd.read_csv("항목2.csv",encoding='utf-8')
항목3 = pd.read_csv("항목3.csv",encoding='utf-8')
항목4 = pd.read_csv("항목4.csv",encoding='utf-8')

In [88]:
building = pd.concat([항목1, 항목2,항목3, 항목4], ignore_index = True)
building = building[['건물건축면적', 'lon','lat','geometry', 'category', '주차면수', '소유주체']]
building.rename(columns={"건물건축면적":"area"}, inplace=True)

In [91]:
building

,area,lon,lat,geometry,category,주차면수,소유주체
0,451.50,127.699952,34.943756,POINT (127.6999517256385 34.94375639192332),주거,61,NaN
1,315.18,127.699701,34.943005,POINT (127.6997008999679 34.94300535627899),주거,61,NaN
2,528.60,127.700178,34.943289,POINT (127.7001775786846 34.94328850811207),주거,61,NaN
3,420.24,127.700143,34.943520,POINT (127.7001428684157 34.94351960690734),주거,61,NaN
4,420.24,127.700320,34.943005,POINT (127.7003199164866 34.94300525165077),주거,61,NaN
...,...,...,...,...,...,...,...
627,79.95,127.768449,34.985126,POINT (127.7684494316839 34.98512629448562),휴게소,0,법인
628,48.00,127.768300,34.984707,POINT (127.7682996761106 34.98470652728316),휴게소,0,법인
629,77.49,127.769068,34.985767,POINT (127.769068141161 34.98576670883541),휴게소,0,법인
630,54.00,127.769183,34.985841,POINT (127.7691831119503 34.98584074102382),휴게소,0,국유지


In [90]:
building.to_csv("광양시_건물정보_전처리.csv", index=False, encoding='cp949')